In [1]:
#25/09/2024 active Final
#Scrapping redbus data using selenium
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  
all_data = []
unique_data = set() 

def scroll_and_load(driver, wait_time=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def scrape_page():
    routes = driver.find_elements(By.XPATH, '//a[@class="route"]')
    
    for index in range(len(routes)):
        try:
            
            routes = driver.find_elements(By.XPATH, '//a[@class="route"]')
            
            
            routename = routes[index].text
            
            
            routelink = routes[index].get_attribute('href')
            
            
            if (routename, routelink) not in unique_data:
                
                unique_data.add((routename, routelink))
                
                
                all_data.append({'routename': routename, 'routelink': routelink})
        
        except Exception as e:
            print(f"An error occurred: {e}")
            continue

urls = [
    "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/north-bengal-state-transport-corporation",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/jksrtc",
    "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
]

for url in urls:
    driver.get(url)
    scroll_and_load(driver, wait_time=3)
    page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
    if page_tabs:
        
        num_tabs = len(page_tabs)
        for i in range(num_tabs):
            page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")        
            if i == 0:
                scrape_page()  
            else:
                page_tab = page_tabs[i]
                
                driver.execute_script("arguments[0].scrollIntoView(true);", page_tab)
                
                driver.execute_script("arguments[0].click();", page_tab)
                
                scroll_and_load(driver, wait_time=3)
                
                scrape_page()
                
                time.sleep(2)
    else:
        scrape_page()

# Print the unique all_data
print(all_data)

def scroll_and_load_elements(routename, routelink):
    bus_records = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        buses = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
        bus_types_elems = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
        dep_times_elems = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
        duration_elems = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
        reaching_time_elems = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
        rating_elems = driver.find_elements(By.CSS_SELECTOR, ".lh-18.rating.rat-orange")
        price_elems = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")
        seat_availa = driver.find_elements(By.CSS_SELECTOR, ".seat-left.m-top-16")

        
        num_buses = len(buses)

        if num_buses == 0:
            print(f"No bus records found on {routelink}")
            return []

        
        for i in range(num_buses):
            bus_record = {
                'bus_name': buses[i].text if i < len(buses) else "N/A",
                'bus_type': bus_types_elems[i].text if i < len(bus_types_elems) else "N/A",
                'departing_time': dep_times_elems[i].text if i < len(dep_times_elems) else "N/A",
                'duration': duration_elems[i].text if i < len(duration_elems) else "N/A",
                'reaching_time': reaching_time_elems[i].text if i < len(reaching_time_elems) else "N/A",
                'rating': rating_elems[i].text if i < len(rating_elems) else "N/A",
                'price': price_elems[i].text if i < len(price_elems) else "N/A",
                'seat_availability': seat_availa[i].text if i < len(seat_availa) else "0",  # Default to 0 seats if not found
                'routename': routename,
                'routelink': routelink
            }
            bus_records.append(bus_record)

       
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return bus_records


bus_data = []
unique_bus_data = set()


for route in all_data[:]:  
    try:
        routename = route['routename']
        routelink = route['routelink']
        
        # Open the route link
        driver.get(routelink)
        time.sleep(5)
        
        # clicking buttons
        try:
            buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='button']")))
            if buttons:
                for button in buttons:
                    try:
                        driver.execute_script("arguments[0].scrollIntoView(true);", button)
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(button))
                        button.click()
                        time.sleep(5)
                    except ElementClickInterceptedException:
                        driver.execute_script("arguments[0].click();", button)
        except TimeoutException:
            print(f"No buttons found on {routelink}, proceeding with scraping.")
        
        # Scrape bus records
        bus_records = scroll_and_load_elements(routename, routelink)
        
        # Add unique bus records
        for record in bus_records:
            bus_tuple = (
                record['bus_name'],
                record['bus_type'],
                record['departing_time'],
                record['duration'],
                record['reaching_time'],
                record['price'],
                record['seat_availability']
            )
            if bus_tuple not in unique_bus_data:
                unique_bus_data.add(bus_tuple)
                bus_data.append(record)

    except Exception as e:
        print(f"An error occurred while processing {routelink}: {e}")


driver.quit()


print("Scraping complete.")
print(f"Total unique routes scraped: {len(all_data)}")
print(f"Total unique bus records scraped: {len(bus_data)}")


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00007FF65DDE9412+29090]
	(No symbol) [0x00007FF65DD5E239]
	(No symbol) [0x00007FF65DC1B1DA]
	(No symbol) [0x00007FF65DBEFAF5]
	(No symbol) [0x00007FF65DC9E2C7]
	(No symbol) [0x00007FF65DCB5EB1]
	(No symbol) [0x00007FF65DC96493]
	(No symbol) [0x00007FF65DC609D1]
	(No symbol) [0x00007FF65DC61B31]
	GetHandleVerifier [0x00007FF65E10871D+3302573]
	GetHandleVerifier [0x00007FF65E154243+3612627]
	GetHandleVerifier [0x00007FF65E14A417+3572135]
	GetHandleVerifier [0x00007FF65DEA5EB6+801862]
	(No symbol) [0x00007FF65DD6945F]
	(No symbol) [0x00007FF65DD64FB4]
	(No symbol) [0x00007FF65DD65140]
	(No symbol) [0x00007FF65DD5461F]
	BaseThreadInitThunk [0x00007FFCDAA6257D+29]
	RtlUserThreadStart [0x00007FFCDBC8AF28+40]


In [ ]:
import pandas as pd
df = pd.DataFrame(bus_data)
df.to_csv('bus_data_8420.csv', index=False)

print(f"Unique data has been saved to unique_scraped_routes.csv. Total unique rows: {len(df)}")

In [ ]:
#create the table in MySql
import mysql.connector

conn = mysql.connector.connect(
    host='localhost',      
    user='root', 
    password='',  
    database='redbus'  
)

cursor = conn.cursor()

# Create the table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bus_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        busname TEXT,
        bustype TEXT,
        departing_time TIME,
        duration TEXT,
        reaching_time TIME,
        star_rating FLOAT,
        price DECIMAL(10, 2),
        seats_available INT
    )
''')


conn.commit()

for record in bus_data:
    price_str = record['price'].replace('₹', '').replace('INR', '').replace(',', '').strip()
    
    try:
        price_value = float(price_str)  # Convert cleaned price string to float
    except ValueError:
        print(f"Could not convert price to float: {record['price']}")
        price_value = None

    # convert of star rating to float
    rating_str = record['rating'].split()[0]  # Extract the rating value
    try:
        rating_value = float(rating_str)
    except ValueError:
        print(f"Could not convert rating to float: {record['rating']}")
        rating_value = None  

    # Insert data into MySQL table
    cursor.execute('''
        INSERT INTO bus_data (
            route_name, route_link, busname, bustype, departing_time, 
            duration, reaching_time, star_rating, price, seats_available
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        record['routename'],
        record['routelink'],
        record['bus_name'],
        record['bus_type'],
        record['departing_time'],
        record['duration'],
        record['reaching_time'],
        rating_value,  
        price_value,  
        int(record['seat_availability'].split()[0])  
    ))

conn.commit()


cursor.close()
conn.close()
